In [23]:
# Import Splinter, BeautifulSoup, and Pandas
from splinter import Browser
from bs4 import BeautifulSoup as soup
import pandas as pd
import re
from webdriver_manager.chrome import ChromeDriverManager

In [24]:
# Declare Global Variables
extract = []
title_list = []
image_list = []
counter=1

In [25]:
# Set the executable path and initialize Splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 92.0.4515
Get LATEST driver version for 92.0.4515
Driver [/Users/stanmisina/.wdm/drivers/chromedriver/mac64/92.0.4515.107/chromedriver] found in cache


### Visit the NASA Mars News Site

In [4]:
# Visit the mars nasa news site
url = 'https://redplanetscience.com/'
browser.visit(url)

# Optional delay for loading the page
browser.is_element_present_by_css('div.list_text', wait_time=1)

True

In [5]:
# Convert the browser html to a soup object and then quit the browser
html = browser.html
news_soup = soup(html, 'html.parser')

slide_elem = news_soup.select_one('div.list_text')

In [6]:
slide_elem.find('div', class_='content_title')

<div class="content_title">The Man Who Wanted to Fly on Mars</div>

In [7]:
# Use the parent element to find the first a tag and save it as `news_title`
news_title = slide_elem.find('div', class_='content_title').get_text()
news_title

'The Man Who Wanted to Fly on Mars'

In [8]:
# Use the parent element to find the paragraph text
news_p = slide_elem.find('div', class_='article_teaser_body').get_text()
news_p

"The Mars Helicopter is riding to the Red Planet this summer with NASA's Perseverance rover. The helicopter's chief engineer, Bob Balaram, shares the saga of how it came into being."

### JPL Space Images Featured Image

In [9]:
# Visit URL
url = 'https://spaceimages-mars.com'
browser.visit(url)

In [10]:
# Find and click the full image button
full_image_elem = browser.find_by_tag('button')[1]
full_image_elem.click()

In [11]:
# Parse the resulting html with soup
html = browser.html
img_soup = soup(html, 'html.parser')
# img_soup

In [12]:
# find the relative image url
img_url_rel = img_soup.find('img', class_='fancybox-image').get('src')
# img_url_rel

In [13]:
# Use the base url to create an absolute url
img_url = f'https://spaceimages-mars.com/{img_url_rel}'
# img_url

### Mars Facts

In [14]:
df = pd.read_html('https://galaxyfacts-mars.com')[0]
df.head()

,0,1,2
0,Mars - Earth Comparison,Mars,Earth
1,Diameter:,"6,779 km","12,742 km"
2,Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
3,Moons:,2,1
4,Distance from Sun:,"227,943,824 km","149,598,262 km"


In [15]:
df.columns=['Description', 'Mars', 'Earth']
df.set_index('Description', inplace=True)
df

,Mars,Earth
Description,,
Mars - Earth Comparison,Mars,Earth
Diameter:,"6,779 km","12,742 km"
Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
Moons:,2,1
Distance from Sun:,"227,943,824 km","149,598,262 km"
Length of Year:,687 Earth days,365.24 days
Temperature:,-87 to -5 °C,-88 to 58°C


In [16]:
# df.to_html()

# D1: Scrape High-Resolution Mars’ Hemisphere Images and Titles

### Hemispheres

In [43]:
# 1. Use browser to visit the URL 
url = 'https://marshemispheres.com/'

browser.visit(url)

In [44]:
# 2. Create a list to hold the images and titles.
hemisphere_image_urls = []


# 3. Write code to retrieve the image urls and titles for each hemisphere.
html = browser.html
hemi_soup = soup(html, 'html.parser')
# select everything in the items list: contains all needed page links and titles
extract = hemi_soup.find_all('div', class_='item')
# extract

In [45]:
# titles are found under div class desription, under as <h3 />
title_list = sorted(re.findall(r'(?<=<h3>)(.*)(?=<\/h3)',str(extract)))

# Images are found from home -> follow hemi page link -> find '<href='name'
# added for navigation url + the extracted page name + added back .html
# image_list_page finds all instances of the string, converts that list to dict to eliminate dups
# then converted back to list - both lists are sorted alphabetically
image_list_page = sorted(list(set(re.findall(r'(?<=href=\")(.*)(?=\.html\")',str(extract)))))
image_list_page = [url + x + '.html' for x in image_list_page]

print(title_list)
print(image_list_page)


['Cerberus Hemisphere Enhanced', 'Schiaparelli Hemisphere Enhanced', 'Syrtis Major Hemisphere Enhanced', 'Valles Marineris Hemisphere Enhanced']
['https://marshemispheres.com/cerberus.html', 'https://marshemispheres.com/schiaparelli.html', 'https://marshemispheres.com/syrtis.html', 'https://marshemispheres.com/valles.html']


In [ ]:
# navigate to hemi pages to collect full size image .jpg address
for page in image_list_page:
    browser.visit(page)
    temp_soup = soup(html, 'html.parser')
    temp_ext = temp_soup.find all('div', class_='')
    


In [20]:
# 4. Print the list that holds the dictionary of each image url and title.
hemisphere_image_urls

[]

[{'img_url': 'https://marshemispheres.com/images/full.jpg',
  'title': 'Cerberus Hemisphere Enhanced'},
 {'img_url': 'https://marshemispheres.com/images/schiaparelli_enhanced-full.jpg',
  'title': 'Schiaparelli Hemisphere Enhanced'},
 {'img_url': 'https://marshemispheres.com/images/syrtis_major_enhanced-full.jpg',
  'title': 'Syrtis Major Hemisphere Enhanced'},
 {'img_url': 'https://marshemispheres.com/images/valles_marineris_enhanced-full.jpg',
  'title': 'Valles Marineris Hemisphere Enhanced'}]

In [21]:
# 5. Quit the browser
browser.quit()